In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
from py_files.Limericks import Limerick_Generate
from py_files.templates import *

import re
import random

In [152]:
def clean_corpus(file):
    corpus = open(file, encoding='latin1')
    raw_text = corpus.read()
    
    processed_text = re.sub(r'[0-9]', r'', raw_text)
    processed_text = re.sub(r'\n\n', r' ', processed_text)
    processed_text = re.sub(r'[.,?!;:]', r'', processed_text).lower().split()
    corpus.close()
    return processed_text

def train_on_corpus(processed_text):
    chain = {}
    for i in range(1, len(processed_text)):
        key = processed_text[i - 1]
        token = processed_text[i]
        if key not in chain:
            chain[key] = []
        chain[key].append(token)
    return chain

In [153]:
all_tokens = [*clean_corpus('data/all_combined/input.txt'), *clean_corpus('data/limericks.txt')]
forward_chain = train_on_corpus(all_tokens)

In [154]:
all_tokens.reverse()
backward_chain = train_on_corpus(all_tokens)

In [83]:
lg = Limerick_Generate()

In [165]:
def fill_word_pos(chain, pos, prev):
    if pos in ['.', ',']:
        return pos
    i = 0
    if prev is None or prev is '[EMPTY]':
        word = random.choice(list(chain.keys()))
        while pos not in lg.words_to_pos[word] and i < 100:
            word = random.choice(list(chain.keys()))
            i += 1
        if i >= 100:
            word = random.choice(lg.pos_to_words[pos])
        return word
    # o/w has a previous word
    word = random.choice(chain[prev])
    while pos not in lg.words_to_pos[word] and i < 100:
        word = random.choice(chain[prev])
        i += 1
    if i >= 100:
        word = random.choice(lg.pos_to_words[pos])
    return word
        
def fill_forward(template):
    prev_word = None
    line = []
    for pos in template:
        line.append(fill_word_pos(forward_chain, pos, prev_word))
        prev_word = line[-1]
    return line
    
def fill_rand_templates(template, num_gen=10):
    for i in range(num_gen):
        line = fill_forward(t)
        print("*****************")
        print(line)
        print(t)

In [80]:
l = random.choice(list(dataset.values()))
t, og = random.choice(l)
print(og)
fill_rand_templates(t)

['oh', 'how', 'modern', '!', 'the', 'critics', 'will', 'say']
*****************
["'scape", 'us-', 'o', 'you', 'any', 'images', 'from', 'mount']
['UH', 'WRB', 'JJ', '.', 'DT', 'NNS', 'MD', 'VB']
*****************
['outstretching', 'of', 'singular', 'sweetness', 'speak', 'words', 'would', 'know']
['UH', 'WRB', 'JJ', '.', 'DT', 'NNS', 'MD', 'VB']
*****************
["to't", "i'd", 'bubble', 'of', 'the', 'ages', 'hence:', 'yet']
['UH', 'WRB', 'JJ', '.', 'DT', 'NNS', 'MD', 'VB']
*****************
['pasture', 'where', 'thou', 'freest', 'waters', 'animals', 'can', 'wait']
['UH', 'WRB', 'JJ', '.', 'DT', 'NNS', 'MD', 'VB']
*****************
['renovates', 'and', 'flat', 'leather', 'that', 'lies', 'would', 'come']
['UH', 'WRB', 'JJ', '.', 'DT', 'NNS', 'MD', 'VB']
*****************
['given', 'up', 'hoarse', 'voice', 'that', 'others', 'will', 'see']
['UH', 'WRB', 'JJ', '.', 'DT', 'NNS', 'MD', 'VB']
*****************
['them\x97old', "age's", 'lambent', 'peaks', 'the', 'bushes', 'at', 'risk']
['UH', '

In [86]:
first_templates = get_first_line()
dataset, second_line, third_line, last_two = get_templates()
templates = {
    1: first_templates,
    2: second_line,
    3: third_line,
    4: last_two
}

In [190]:
def fill_backward(template, prev_word=None):
    if prev_word:
        line = [prev_word]
    else:
        line = []
    i = len(template) - len(line) - 1
    while i >= 0:
        pos = template[i]
        line.insert(0, fill_word_pos(backward_chain, pos, prev_word))
        if line[0] not in ['.', ',']:
            prev_word = line[0]
        i -= 1
    return line

def gen_poem(five_words):
    poem = []
    for i in range(3):
        word_pos = lg.words_to_pos[five_words[i]][0]
        t, og = random.choice(templates[i + 1][word_pos])
        poem.append((fill_backward(t, prev_word=five_words[i]), t, og))
    last_two_pos = lg.words_to_pos[five_words[3]][0] + '-' + lg.words_to_pos[five_words[4]][0]
    t, og, idx = random.choice(templates[4][last_two_pos])
    poem.append((fill_backward(t[:idx + 1], prev_word=five_words[3]), t[:idx + 1], og[:idx + 1]))
    poem.append((fill_backward(t[idx + 1:], prev_word=five_words[4]), t[idx + 1:], og[idx + 1:]))
    return poem

In [187]:
lg.get_five_words('face')

('thrace', 'face', 'report', 'court', 'case')

In [186]:
lg.words_to_pos[''].remove('NNP')
lg.pos_to_words['NNP'].remove('birthplace')

In [136]:
def print_poem(five_words):
    for line, template, og in gen_poem(five_words):
        print(' '.join(line))
        print(template)
        print(og)
        print('**********************')

In [166]:
print_poem(('greece', 'peace', 'mind', 'kind', 'piece'))

all right fair crown of greece
['DT', 'JJ', 'JJ', 'NN', 'IN', 'NNP']
['a', 'silly', 'young', 'man', 'from', 'clyde']
**********************
who was the lack of peace
['WHO', 'VBD', 'DT', 'NN', 'IN', 'NN']
['who', 'hadnt', 'an', 'atom', 'of', 'fear']
**********************
both some brigades inclined mind
['PDT', 'DT', 'NNS', 'VBD', 'IN']
['all', 'the', 'flowers', 'looked', 'around']
**********************
these briar and a kind
['DT', 'NN', 'CC', 'DT', 'NN']
['a', 'howl', 'and', 'a', 'squeak']
**********************
but your sawyer had only piece
['CC', 'PRP$', 'NN', 'VBD', 'RB', 'JJ']
['and', 'his', 'language', 'was', 'really', 'good']
**********************


In [168]:
print_poem(('dane', 'gain', 'decrease', 'increase', 'vein'))

that mine whatever found the dane
['DT', 'NN', 'WP', 'VBD', 'DT', 'NN']
['a', 'tutor', 'who', 'tooted', 'a', 'flute']
**********************
who dug herself gaily of the gain
['WHO', 'VBD', 'PRP', 'RB', 'IN', 'DT', 'NN']
['who', 'found', 'himself', 'quite', 'at', 'a', 'loss']
**********************
both another subjects wrung decrease
['PDT', 'DT', 'NNS', 'VBD', 'IN']
['all', 'the', 'flowers', 'looked', 'around']
**********************
that came to its increase
['WDT', 'VBD', 'TO', 'PRP$', 'NN']
['which', 'came', 'to', 'his', 'aid']
**********************
took long the mossy vein
['VBD', 'RB', 'DT', 'JJ', 'NN']
['found', 'only', 'a', 'sleeveless', 'tuxedo']
**********************


In [191]:
print_poem(('thrace', 'face', 'report', 'court', 'case'))

there lived a long cedar choked thrace
['EX', 'VBD', 'DT', 'JJ', 'NN', 'VBN', 'NNP']
['there', 'was', 'a', 'young', 'lady', 'named', 'rose']
**********************
who made of golden face
['WHO', 'VBD', 'IN', 'JJ', 'NN']
['who', 'lived', 'on', 'distilled', 'kerosene']
**********************
the ruin upon your report
['DT', 'NN', 'IN', 'PRP$', 'NN']
['the', 'cause', 'of', 'his', 'sorrow']
**********************
ruffled every word in our court
['VBD', 'DT', 'NN', 'IN', 'PRP$', 'NNS']
['wet', 'the', 'bloom', 'on', 'her', 'cheeks']
**********************
and they were witty and turned our case
['CC', 'PRP', 'VBD', 'RB', 'CC', 'VBD', 'PRP$', 'NN']
['and', 'it', 'dropped', 'down', 'and', 'ruined', 'her', 'gown']
**********************


In [127]:
with open('data/all_combined/input.txt', encoding='latin1') as f:
    for i, line in enumerate(f):
        if i == 40841:
            print(line)

The course of Time and nationsEgypt, India, Greece and 



In [197]:
import pickle

with open('py_files/saved_objects/postag_dict_all.p', 'rb') as f:
    postag = pickle.load(f)

postag[1] = lg.pos_to_words
postag[2] = lg.words_to_pos

with open('py_files/saved_objects/postag_dict_all.p', 'wb') as f:
    pickle.dump(postag, f)